<a href="https://colab.research.google.com/github/ivnlee/tec-mna-nlp/blob/main/semana-9/Equipo13_semana09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Modelos Transformer para textos en Español e Inglés

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers
!pip install datasets
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 92.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 72.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 1.6 MB/s eta 0:00:00


In [3]:
import transformers

#Set to avoid warning messages.
transformers.logging.set_verbosity_error()

### **1. Dataset amazon-baby5000**

In [4]:
from datasets import load_dataset

text = load_dataset('csv', data_files='/content/drive/MyDrive/NLP/semana-9/amazonbaby5000.csv')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-f61dc3b5d8005f58/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
text

DatasetDict({
    train: Dataset({
        features: ['review', 'rating'],
        num_rows: 5000
    })
})

In [ ]:
# convertimos a un dataframe de pandas para observar las primeras filas
text.set_format(type='pandas')
df = text['train'][:]

df.head()

,review,rating
0,I think it is more Expensive than drugstore th...,0
1,"When I saw this on Amazon, I put it into my wi...",1
2,We really like these valances. They have such...,1
3,No light emits from the night light. They pain...,0
4,I was really hoping for this to be a conventie...,0


In [ ]:
# Observamos la distribucion de las clases
df['rating'].value_counts()

1    3840
0    1160
Name: rating, dtype: int64

### **2. Realiza una partición de los datos en el porcentaje que consideres adecuado, entrenamiento, validación y prueba.**

In [ ]:
from sklearn.model_selection import train_test_split

X = df['review']
Y = df['rating']

# Primer split para el set de entrenamiento 70%, el parametro 'stratify' mantiene la distribucion de clases
X_train, X_t, y_train, y_t = train_test_split(X, Y, test_size=0.3, random_state=42, stratify=Y)

# Segundo split para sets de validación 15% y prueba 15%
X_val, X_test, y_val, y_test = train_test_split(X_t, y_t, test_size=0.5, random_state=42, stratify=y_t)

In [ ]:
print('Distribución de clases del set de Entrenamiento:\n', y_train.value_counts())
print('Distribución de clases del set de Validación:\n', y_val.value_counts())
print('Distribución de clases del set de Prueba:\n', y_test.value_counts())

Distribución de clases del set de Entrenamiento:
 1    2688
0     812
Name: rating, dtype: int64
Distribución de clases del set de Validación:
 1    576
0    174
Name: rating, dtype: int64
Distribución de clases del set de Prueba:
 1    576
0    174
Name: rating, dtype: int64


In [ ]:
text.reset_format()
text

DatasetDict({
    train: Dataset({
        features: ['review', 'rating'],
        num_rows: 5000
    })
})

In [7]:
#text['train'].train_test_split(test_size=0.25)

DatasetDict({
    train: Dataset({
        features: ['review', 'rating'],
        num_rows: 3750
    })
    test: Dataset({
        features: ['review', 'rating'],
        num_rows: 1250
    })
})

### **3. Utiliza un modelo Transformer de HuggingFace para análisis de sentimiento en inglés y lleva a cabo la predicción de los comentarios en los siguientes casos:**

### a. Utilizando un modelo predeterminado de HF de manera directa, es decir, sin ajuste de parámetros del modelo. Indicar el desempeño (accuracy) obtenido.

In [8]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
my_sentences = X_test[0:5].tolist()

In [ ]:
model_name = 'siebert/sentiment-roberta-large-english'

classifier = pipeline("sentiment-analysis",
                        model= model_name,
                        tokenizer= model_name,
                        top_k=None
                        )


In [ ]:
classifier(my_sentences)

[[{'label': 'POSITIVE', 'score': 0.9989227652549744},
  {'label': 'NEGATIVE', 'score': 0.0010772388195618987}],
 [{'label': 'POSITIVE', 'score': 0.9989196062088013},
  {'label': 'NEGATIVE', 'score': 0.0010804433841258287}],
 [{'label': 'POSITIVE', 'score': 0.9989239573478699},
  {'label': 'NEGATIVE', 'score': 0.0010759617434814572}],
 [{'label': 'NEGATIVE', 'score': 0.9995108842849731},
  {'label': 'POSITIVE', 'score': 0.0004891530843451619}],
 [{'label': 'NEGATIVE', 'score': 0.9995101690292358},
  {'label': 'POSITIVE', 'score': 0.0004898576298728585}]]

In [ ]:
y_test[0:5]

1142    1
1654    1
4632    1
1954    0
1597    0
Name: rating, dtype: int64

### b. Utiliza ahora un ajuste de parámetros (fine-tuning) con el modelo Transformer DistilBERT, para generar un nuevo modelo y clasificación de los comentarios. Indicar desempeño (accuracy) obtenido.

In [43]:
import torch
torch.cuda.is_available()

True

In [ ]:
!pip install datasets transformers huggingface_hub

In [11]:
!apt-get install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


In [12]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [19]:
amazon_revs = text['train'].train_test_split(test_size=0.25)

In [23]:
train_data = amazon_revs['train']
test_data = amazon_revs['test']

In [25]:
def preprocess_function(examples):
   return tokenizer(examples["review"], truncation=True)

tokenized_train = train_data.map(preprocess_function, batched=True)
tokenized_test = test_data.map(preprocess_function, batched=True)

Map:   0%|          | 0/3750 [00:00<?, ? examples/s]

Map:   0%|          | 0/1250 [00:00<?, ? examples/s]

In [26]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [27]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

In [28]:
import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")

   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}

In [ ]:
!pip install transformers[torch]

In [39]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!pip install accelerate

In [ ]:
from transformers import TrainingArguments, Trainer

repo_name = "finetuning-sentiment-model-3000-samples"

training_args = TrainingArguments(
    output_dir=repo_name,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    save_strategy="epoch",
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)